In [10]:
import pandas as pd

df = pd.read_csv("restaurant_data.csv")
vd = pd.read_csv("violation_data.csv")

In [ ]:
import pyodbc


ModuleNotFoundError: No module named 'sql_pass'

In [ ]:
%pip install pyodc

In [19]:
import pyodbc as pyodbc
conn_sql = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=johndroescher.com;'
                      'Database=Fall_2022;'
                      'UID=mariaalguacil66;'
                      'PWD=Ccny23424266;')
cnxn = pyodbc.connect(conn_sql)

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [12]:
password = sql_pass
username = "sa"
default_db = "Fall_2022"
server = "localhost"

cnxn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};SERVER="
    + server
    + ";DATABASE="
    + default_db
    + ";UID="
    + username
    + ";PWD="
    + password
)

NameError: name 'sql_pass' is not defined

In [13]:
extract_data = pd.read_sql(
"""SELECT rest.camis
          ,rest.dba
          ,rest.bin
          ,rest.cuisine_description
          ,viol.violation_description
          ,viol.action
          ,viol.score
          ,viol.grade_date
          ,viol.inspection_date
          ,viol.critical_flag
          ,viol.record_date
          
     FROM dbo.restaurant_data rest
          LEFT JOIN dbo.violation_data viol
                    ON rest.camis = viol.camis
""", 
cnxn
)

NameError: name 'cnxn' is not defined

In [14]:
extract_data.shape

NameError: name 'extract_data' is not defined

In [ ]:
extract_data.columns.tolist()

['camis',
 'dba',
 'bin',
 'cuisine_description',
 'violation_description',
 'action',
 'score',
 'grade_date',
 'inspection_date',
 'critical_flag',
 'record_date']

In [ ]:
all_data = extract_data.copy()
all_data["inspection_date"]

NameError: name 'extract_data' is not defined

In [ ]:
all_data["inspection_date"] = pd.to_datetime(all_data["inspection_date"])

In [ ]:
all_data.dtypes

camis                             int64
dba                              object
bin                             float64
cuisine_description              object
violation_description            object
action                           object
score                           float64
grade_date               datetime64[ns]
inspection_date          datetime64[ns]
critical_flag                    object
record_date              datetime64[ns]
dtype: object

In [ ]:
# Create base list that includes camis
col_names = ['camis']

# Loop through column names to create a list of date columns
for col in all_data.columns.to_list():
    if 'date' in col:
        col_names.append(col)

col_names

['camis', 'grade_date', 'inspection_date', 'record_date']

In [ ]:
# List Comprehension that accomplishes what the for loop above accomplishes
col_names_2 = [col for col in all_data.columns.to_list() if ('date' in col) or ('camis' in col)]

In [ ]:
col_names_2

['camis', 'grade_date', 'inspection_date', 'record_date']

In [ ]:
all_data[col_names]

,camis,grade_date,inspection_date,record_date
0,50010393,2019-10-15,2019-10-15,2022-08-27 06:00:12
1,50099700,2020-01-30,2020-01-30,2022-08-27 06:00:12
2,50111874,NaT,2021-09-07,2022-08-27 06:00:12
3,41640776,2018-12-04,2018-12-04,2022-08-27 06:00:12
4,50088892,NaT,2019-09-03,2022-08-27 06:00:12
...,...,...,...,...
248015,50015281,2018-06-04,2018-06-04,2022-08-27 06:00:12
248016,50066680,2019-10-25,2019-10-25,2022-08-27 06:00:12
248017,50045154,NaT,2022-08-22,2022-08-27 06:00:12
248018,50070200,2020-01-28,2020-01-28,2022-08-27 06:00:12


In [ ]:
all_data.sort_values(['camis', 'inspection_date']).groupby('camis')['inspection_date'].last()

NameError: name 'all_data' is not defined

In [ ]:
all_data[col_names].record_date.value_counts()

2022-08-27 06:00:12    243166
2022-08-27 06:00:16      4148
2022-08-27 06:00:14       706
Name: record_date, dtype: int64

In [ ]:
all_data.sort_values(['camis', 'inspection_date']).groupby('camis')['inspection_date'].first()

NameError: name 'all_data' is not defined

In [ ]:
# Create new df for holding max / min inspection dates
inspection_date_df = pd.DataFrame(index=all_data['camis'].unique())

# Add min inspection date by camis
inspection_date_df['min'] = all_data.groupby('camis')['inspection_date'].min()

# Add record date by camis
inspection_date_df['record'] = all_data.groupby('camis')['record_date'].max()

# Add max inspection date by camis
inspection_date_df['max'] = all_data.groupby('camis')['inspection_date'].max()

# Add columns of calculated differeces in date
inspection_date_df['days_open'] = inspection_date_df[['min', 'record']].diff(axis=1)['record']
inspection_date_df['days_since_inspec'] = inspection_date_df[['max', 'record']].diff(axis=1)['record']

In [ ]:
inspection_date_df['date_diff_yrs'] = inspection_date_df.date_diff / dt.timedelta(365)

In [ ]:
inspection_date_df[inspection_date_df.date_diff_yrs < 20]

,min,max,date_diff,date_diff_yrs
50010393,2019-07-25,2022-08-27 06:00:12,1129 days 06:00:12,3.093836
50099700,2019-11-18,2022-08-27 06:00:14,1013 days 06:00:14,2.776028
50111874,2021-09-07,2022-08-27 06:00:12,354 days 06:00:12,0.970548
41640776,2018-12-04,2022-08-27 06:00:12,1362 days 06:00:12,3.732192
50088892,2019-09-03,2022-08-27 06:00:12,1089 days 06:00:12,2.984247
...,...,...,...,...
50110882,2021-08-04,2022-08-27 06:00:12,388 days 06:00:12,1.063699
50107959,2021-11-29,2022-08-27 06:00:12,271 days 06:00:12,0.743151
50100378,2020-02-13,2022-08-27 06:00:12,926 days 06:00:12,2.537672
50117920,2022-01-05,2022-08-27 06:00:12,234 days 06:00:12,0.641781


In [ ]:
inspection_date_df['closed'] = inspection_date_df.days_since_inspec > dt.timedelta(365)

In [ ]:
all_data.merge(inspection_date_df['closed'])

50010393    False
50099700    False
50111874    False
41640776    False
50088892    False
            ...  
50122875     True
50120707     True
50116939     True
50117920    False
50067499    False
Name: closed, Length: 27960, dtype: bool

In [ ]:
inspection_date_df

In [ ]:
no_of_lectures = 30
mins_in_lecture = 75
total_mins = 0

for lecture in range(no_of_lectures):
    total_mins += mins_in_lecture
print(total_mins)

2250


In [ ]:
filter_columns = all_data.filter(items=["camis", "dba", "boro", "inspection_date"])
print(filter_columns)

           camis                                           dba       boro  \
0       50010393                                           NaN        NaN   
1       50099700                                           NaN        NaN   
2       50111874                                           NaN        NaN   
3       41640776                                           NaN        NaN   
4       50088892                                           NaN        NaN   
...          ...                                           ...        ...   
248015  50015281                                MALAYSIA GRILL  Manhattan   
248016  50066680                  ADDICTIVE WINE BAR & TAPAS 1     Queens   
248017  50045154  RIU PLAZA (Fashion Bar & Kitchen, Grab & Go)  Manhattan   
248018  50070200                      SAHARA'S TURKISH CUISINE  Manhattan   
248019  50044140                                BOULE & CHERIE  Manhattan   

                    inspection_date  
0       2019-10-15 00:00:00.0000000  